<a href="https://colab.research.google.com/github/roggersanguzu/Car-price-Predictions/blob/main/Car_price_Predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import matplotlib.pyplot as plt
import matplotlib as matplotlib
import seaborn as sns
import plotly.express as px
import numpy as np
import pandas as pd
%matplotlib inline

import warnings

from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

warnings.filterwarnings('ignore')

print("Libraries imported successfully")

Libraries imported successfully


In [3]:
from google.colab import drive
drive.mount('/content/drive')
file_path="/content/drive/MyDrive/Colab Notebooks/Datasets/AutoDataset.csv"
car_df=pd.read_csv(file_path,encoding='latin1')
car_df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,3/26/2016 17:47,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,3/26/2016 0:00,0,79588,4/6/2016 6:45
1,4/4/2016 13:38,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,4/4/2016 0:00,0,71034,4/6/2016 14:45
2,3/26/2016 18:57,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,3/26/2016 0:00,0,35394,4/6/2016 20:15
3,3/12/2016 16:58,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,3/12/2016 0:00,0,33729,3/15/2016 3:16
4,4/1/2016 14:38,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,4/1/2016 0:00,0,39218,4/1/2016 14:38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,3/27/2016 14:38,Audi_Q5_3.0_TDI_qu._S_tr.__Navi__Panorama__Xenon,privat,Angebot,"$24,900",control,limousine,2011,automatik,239,q5,"100,000km",1,diesel,audi,nein,3/27/2016 0:00,0,82131,4/1/2016 13:47
49996,3/28/2016 10:50,Opel_Astra_F_Cabrio_Bertone_Edition___TÜV_neu+...,privat,Angebot,"$1,980",control,cabrio,1996,manuell,75,astra,"150,000km",5,benzin,opel,nein,3/28/2016 0:00,0,44807,4/2/2016 14:18
49997,4/2/2016 14:44,Fiat_500_C_1.2_Dualogic_Lounge,privat,Angebot,"$13,200",test,cabrio,2014,automatik,69,500,"5,000km",11,benzin,fiat,nein,4/2/2016 0:00,0,73430,4/4/2016 11:47
49998,3/8/2016 19:25,Audi_A3_2.0_TDI_Sportback_Ambition,privat,Angebot,"$22,900",control,kombi,2013,manuell,150,a3,"40,000km",11,diesel,audi,nein,3/8/2016 0:00,0,35683,4/5/2016 16:45


In [4]:
car_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   dateCrawled          50000 non-null  object
 1   name                 50000 non-null  object
 2   seller               50000 non-null  object
 3   offerType            50000 non-null  object
 4   price                50000 non-null  object
 5   abtest               50000 non-null  object
 6   vehicleType          44905 non-null  object
 7   yearOfRegistration   50000 non-null  int64 
 8   gearbox              47320 non-null  object
 9   powerPS              50000 non-null  int64 
 10  model                47242 non-null  object
 11  odometer             50000 non-null  object
 12  monthOfRegistration  50000 non-null  int64 
 13  fuelType             45518 non-null  object
 14  brand                50000 non-null  object
 15  notRepairedDamage    40171 non-null  object
 16  date

In [5]:
car_df.describe()

,yearOfRegistration,powerPS,monthOfRegistration,nrOfPictures,postalCode
count,50000.000000,50000.000000,50000.000000,50000.0,50000.000000
mean,2005.073280,116.355920,5.723360,0.0,50813.627300
std,105.712813,209.216627,3.711984,0.0,25779.747957
min,1000.000000,0.000000,0.000000,0.0,1067.000000
25%,1999.000000,70.000000,3.000000,0.0,30451.000000
50%,2003.000000,105.000000,6.000000,0.0,49577.000000
75%,2008.000000,150.000000,9.000000,0.0,71540.000000
max,9999.000000,17700.000000,12.000000,0.0,99998.000000


**Removing non-numeric characters like currency symbols or km**

In [6]:
car_df['price'] = car_df['price'].str.replace('[^0-9]', '', regex=True).astype(float)
car_df['odometer'] = car_df['odometer'].str.replace('[^0-9]', '', regex=True).astype(float)


In [7]:
car_df = car_df[(car_df['price'] > 100) & (car_df['price'] < 150000)]
car_df = car_df[(car_df['odometer'] > 500) & (car_df['odometer'] < 250000)]

car_df['log_price'] = np.log1p(car_df['price'])

Now price and odometer are numeric.

**Identify features**

In [8]:
features = ['vehicleType','yearOfRegistration','gearbox','powerPS','model','odometer',
            'monthOfRegistration','fuelType','brand','notRepairedDamage']

X = car_df[features]
y = car_df['log_price']


**Split categorical vs numeric**

In [9]:
numeric_feats = X.select_dtypes(include=['int64','float64']).columns.tolist()
categorical_feats = X.select_dtypes(include=['object']).columns.tolist()

print("Numeric features:", numeric_feats)
print("Categorical features:", categorical_feats)


Numeric features: ['yearOfRegistration', 'powerPS', 'odometer', 'monthOfRegistration']
Categorical features: ['vehicleType', 'gearbox', 'model', 'fuelType', 'brand', 'notRepairedDamage']


**Preprocessing pipelines**

In [10]:

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])


categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_feats),
    ('cat', categorical_transformer, categorical_feats)
])


In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


**Choose models and hyperparameter grids**

In [12]:
models = {
    'DecisionTree': DecisionTreeRegressor(random_state=42),
    'RandomForest': RandomForestRegressor(random_state=42, n_jobs=-1),
    'GradientBoosting': GradientBoostingRegressor(random_state=42)
}

param_grids = {
    'DecisionTree': {'clf__max_depth': [8,2], 'clf__min_samples_split':[10,20]},
    'RandomForest': {'clf__n_estimators':[100], 'clf__max_depth':[10,20,None], 'clf__max_features':['sqrt','log2']},
    'GradientBoosting': {'clf__n_estimators':[100,200], 'clf__learning_rate':[0.01,0.1], 'clf__max_depth':[3,5]}
}


**Train Models with GridSearchCV, In Our Group we decided to randomise the search instead of evaluating the full grid **

In [14]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error

best_estimators = {}

for name, model in models.items():
    print(f"\n--- Hyperparameter Search for {name} ---")


    pipeline = Pipeline([('pre', preprocessor), ('reg', model)])

    cv = KFold(n_splits=3, shuffle=True, random_state=42)

    grid = RandomizedSearchCV(
        pipeline,
        param_distributions={
            k.replace('clf__', 'reg__'): v for k, v in param_grids[name].items()
        },
        n_iter=5,
        cv=cv,
        scoring='r2',
        n_jobs=-1,
        verbose=1,
        random_state=42
    )

    grid.fit(X_train, y_train)

    print("Best params:", grid.best_params_)
    print("Best CV R2:", grid.best_score_)

    y_pred = grid.predict(X_test)
    test_r2 = r2_score(y_test, y_pred)
    test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    print("Test R2:", test_r2)
    print("Test RMSE:", test_rmse)


    best_estimators[name] = grid.best_estimator_



--- Hyperparameter Search for DecisionTree ---
Fitting 3 folds for each of 4 candidates, totalling 12 fits
Best params: {'reg__min_samples_split': 10, 'reg__max_depth': 8}
Best CV R2: 0.7481986101258299
Test R2: 0.7519885321379092
Test RMSE: 0.5990513122884784

--- Hyperparameter Search for RandomForest ---
Fitting 3 folds for each of 5 candidates, totalling 15 fits
Best params: {'reg__n_estimators': 100, 'reg__max_features': 'sqrt', 'reg__max_depth': None}
Best CV R2: 0.8145644347296948
Test R2: 0.8201307888500187
Test RMSE: 0.5101601803345297

--- Hyperparameter Search for GradientBoosting ---
Fitting 3 folds for each of 5 candidates, totalling 15 fits
Best params: {'reg__n_estimators': 200, 'reg__max_depth': 5, 'reg__learning_rate': 0.1}
Best CV R2: 0.8337774366074076
Test R2: 0.8363174879377635
Test RMSE: 0.4866640694161897
